### [Convert Sorted List into a Binary Search Tree](https://leetcode.com/problems/convert-sorted-list-to-binary-search-tree/description/)

Given a singly linked list where elements are sorted in ascending order, convert it to a height balanced BST.

For this problem, a height-balanced binary tree is defined as a binary tree in which the depth of the two subtrees of every node never differ by more than 1.

Example:
```
Given the sorted linked list: [-10,-3,0,5,9],

One possible answer is: [0,-3,9,-10,null,5], which represents the following height balanced BST:

      0
     / \
   -3   9
   /   /
 -10  5
 ```

In [20]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

# Definition for a binary tree node.
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

class Solution:
    def sortedListToBST(self, head):
        """
        :type head: ListNode
        :rtype: TreeNode
        """
        return self.sortedListToBSTUsingList(head)
        
    def sortedListToBSTUsingList(self, head):
        """
        :type head: ListNode
        :rtype: TreeNode
        """
        # walking the list directly this time
        # find the root node in the list using two pointer method. 
        # keep track of the previous node of the root to cut the
        # list into two sub lists.
        if not head:
            return None
        
        prev = slow = fast = None
        
        # Steps to find the root is halved at each iteration
        slow = fast = head
        while fast and fast.next:
            prev = slow
            slow = slow.next
            fast = fast.next.next
        
        # in case of single element list, there won't be any prev
        # so no need to separate cut the list on the left.
        if prev:
            prev.next = None
            
        leftList = head if slow != head else None
        rightList = slow.next
        
        root = TreeNode(slow.val)
        root.left = self.sortedListToBST(leftList)
        root.right = self.sortedListToBST(rightList)
        
        return root
        
    def sortedListToBSTUsingArrayConversion(self, head):
        """
        :type head: ListNode
        :rtype: TreeNode
        """
        # height balanced
        # list in sorted order.. 1st node is the left most in the left subtree
        # pick the middle as the root.. 0...root goes to the left subtree
        # root+1...right goes to the right subtree
        # inorder traversal to binary tree
        # but input here is not an array.. instead it is a list
        # so searching for the middle will be n/2 ops.
        # have to walk the list anyway to fetch the root value.
        # can compute the length once.. O(n) hit
        # root = n/2.. walk the list n/2 steps.  if input list can be modified,
        # we can cut the sortedlist into two sublists
        # 
        # trade some space for time?? convert sorted list into array
        # and then build the tree. O(n) space, O(n) time. 
        
        # Convert sorted list into a sorted array
        sortedVals = []
        node = head
        while node:
            sortedVals.append(node.val)
            node = node.next
        
        def worker(values):
            if not values:
                return None
            
            rootIndex = len(values) // 2
            root = TreeNode(values[rootIndex])
            root.left = worker(values[:rootIndex])
            root.right = worker(values[rootIndex+1:])
            
            return root
        
        return worker(sortedVals)
    
    def sortedListToBSTUsingArrayConversionWithIndex(self, head):
        """
        :type head: ListNode
        :rtype: TreeNode
        """
        # Pretty much as same sortedListToBSTUsingArrayConversion, except
        # that it passes the start and end indexes instead of creating a
        # new list on each call. List subscripting creates a shallow copy
        # of the list only, so not very expensive. But it is better to
        # avoid when it is not needed. Also, I wanted to be little more
        # comfortable with this type of solutions as well.
        
        # Convert sorted list into a sorted array
        sortedVals = []
        node = head
        while node:
            sortedVals.append(node.val)
            node = node.next
        
        def worker(values, start, end):
            
            if not values:
                return None
            
            if start > end:
                return None
            
            if start == end:
                return TreeNode(values[start])
            
            rootIndex = start + ((end - start) // 2)
            root = TreeNode(values[rootIndex])
            root.left = worker(values, start, rootIndex - 1)
            root.right = worker(values, rootIndex + 1, end)
            
            return root
        
        return worker(sortedVals, 0, len(sortedVals) - 1)
            

Run the solution through some tests

In [26]:
import random

def generatateTestLists(numLists):
    """ Generate test lists to validate the conversion of sorted linked list to BST """
    
    def listToLinkedList(valList):
        """ Convert a python list into linked list of ListNode objects"""
        if not valList:
            return None
        
        head = ListNode(valList[0])
        node = head
        for i in range(1, len(valList)):
            node.next = ListNode(valList[i])
            node = node.next
        
        return head
        
        
    listSizes = [random.randrange(0, 100) for _ in range(numLists)]
    testLists = []
    for i in range(numLists):
        listLength = random.randrange(0, 100)
        testList = [random.randrange(1000) for _ in range(listLength)]
        testList = list(set(testList)) # Removing duplicates
        testList.sort()
        testListHead = listToLinkedList(testList)
        testLists.append(testListHead)

    
    return testLists

def inOrder(root):
    if root:
        yield from inOrder(root.left)
        yield root.val
        yield from inOrder(root.right)

testLists = generatateTestLists(numLists=5)
s = Solution()
for i in range(len(testLists)):
    testListInBST = s.sortedListToBST(testLists[i])
    valuesInTestListBST = list(inOrder(testListInBST))
    
    # if the tree was correctly built, values collected in
    # in-order traversal should be in sorted order
    # print("Testing tree#{0} in in-order gives {1}\n".format(i, valuesInTestListBST))
    assert valuesInTestListBST == sorted(valuesInTestListBST)
    


Coming to some analysis. 

* sortedListToBSTUsingArrayConversion() - runs in O(n) space and O(n) time
* sortedListToBSTUsingList() - runs in O(1) space and O(n * log(n)) time. Internal worker() function will be called once for every node in the list. On each iteration, cost of finding the root will be halved as we walk up to only the middle node.
* ArrayConversion solution ran faster with LeetCode OJ. In real applications, we have to choose based on the application requirements though.